## Geocoding

In [1]:
import pandas as pd
import geopandas as gpd
import geopy
from geopy.geocoders import Nominatim, Here
from geopy.extra.rate_limiter import RateLimiter
import matplotlib.pyplot as plt
import folium
from folium.plugins import FastMarkerCluster

In [2]:
here_app_id = "V120bIMd1Cc3snPH5s2p"
here_api_key= "N-ij6s0myoykwYbIGhRRbUs1oNeyIWdn_o635gXyxwo"

### Geocoding Single Address 

In [3]:
#locator = Nominatim(user_agent="myGeocoder")
locator = Here(apikey= here_api_key, app_id=here_app_id)
location = locator.geocode("Champ de Mars, Paris, France")

In [4]:
print(location.address)
print("Latitude = {}, Longitude = {}".format(location.latitude, location.longitude))

Champ-de-Mars, 75007 Paris, France, Paris, Île-de-France 75007, FRA
Latitude = 48.85478, Longitude = 2.30038


### Geocoding with Pandas DataFrame

In [5]:
header_list = ["TUID", "Price", "Date", "Postcode", "Property_Type", "OldNew", "Duration", "PAON","SAON","Street", "Locality", "City", "District", "County", "PPDtype", "RecordStatus"]
df = pd.read_csv('C:/Users/METE/Downloads/pp-monthly-update-new-version.csv', names=header_list)
df.head()

,TUID,Price,Date,Postcode,Property_Type,OldNew,Duration,PAON,SAON,Street,Locality,City,District,County,PPDtype,RecordStatus
0,{C6209F5F-241F-295E-E053-6C04A8C0DDCC},310000,2004-09-08 00:00,RM12 4DP,T,N,F,51,NaN,BANCROFT CHASE,NaN,HORNCHURCH,HAVERING,GREATER LONDON,A,A
1,{C6209F5F-4752-295E-E053-6C04A8C0DDCC},440000,2004-04-08 00:00,WD25 0DY,D,N,F,22,NaN,SHEEPCOT DRIVE,NaN,WATFORD,WATFORD,HERTFORDSHIRE,A,A
2,{C6209F5F-497D-295E-E053-6C04A8C0DDCC},415000,2004-03-26 00:00,AL1 1NL,T,N,F,6,NaN,RYDER SEED MEWS,NaN,ST ALBANS,ST ALBANS,HERTFORDSHIRE,A,A
3,{C6209F5F-9679-295E-E053-6C04A8C0DDCC},540000,2004-10-12 00:00,SW7 4DA,F,N,L,29 - 31,FLAT 8,COURTFIELD ROAD,NaN,LONDON,KENSINGTON AND CHELSEA,GREATER LONDON,A,A
4,{C6209F5F-C5BA-295E-E053-6C04A8C0DDCC},570000,2004-08-12 00:00,IP14 3JH,D,N,F,NELSONS,NaN,MOORBRIDGE LANE,HARLESTON,STOWMARKET,MID SUFFOLK,SUFFOLK,A,A


In [6]:
df['Country'] = 'United Kingdom'
#Concatenate strings across columns that are not null
df['Address'] = df[['PAON', 'SAON', 'Street', 'Locality', 'City', 'District', 'Postcode', 'County', 'Country']].apply(lambda x: ','.join(x.dropna()), axis=1)
df

,TUID,Price,Date,Postcode,Property_Type,OldNew,Duration,PAON,SAON,Street,Locality,City,District,County,PPDtype,RecordStatus,Country,Address
0,{C6209F5F-241F-295E-E053-6C04A8C0DDCC},310000,2004-09-08 00:00,RM12 4DP,T,N,F,51,NaN,BANCROFT CHASE,NaN,HORNCHURCH,HAVERING,GREATER LONDON,A,A,United Kingdom,"51,BANCROFT CHASE,HORNCHURCH,HAVERING,RM12 4DP..."
1,{C6209F5F-4752-295E-E053-6C04A8C0DDCC},440000,2004-04-08 00:00,WD25 0DY,D,N,F,22,NaN,SHEEPCOT DRIVE,NaN,WATFORD,WATFORD,HERTFORDSHIRE,A,A,United Kingdom,"22,SHEEPCOT DRIVE,WATFORD,WATFORD,WD25 0DY,HER..."
2,{C6209F5F-497D-295E-E053-6C04A8C0DDCC},415000,2004-03-26 00:00,AL1 1NL,T,N,F,6,NaN,RYDER SEED MEWS,NaN,ST ALBANS,ST ALBANS,HERTFORDSHIRE,A,A,United Kingdom,"6,RYDER SEED MEWS,ST ALBANS,ST ALBANS,AL1 1NL,..."
3,{C6209F5F-9679-295E-E053-6C04A8C0DDCC},540000,2004-10-12 00:00,SW7 4DA,F,N,L,29 - 31,FLAT 8,COURTFIELD ROAD,NaN,LONDON,KENSINGTON AND CHELSEA,GREATER LONDON,A,A,United Kingdom,"29 - 31,FLAT 8,COURTFIELD ROAD,LONDON,KENSINGT..."
4,{C6209F5F-C5BA-295E-E053-6C04A8C0DDCC},570000,2004-08-12 00:00,IP14 3JH,D,N,F,NELSONS,NaN,MOORBRIDGE LANE,HARLESTON,STOWMARKET,MID SUFFOLK,SUFFOLK,A,A,United Kingdom,"NELSONS,MOORBRIDGE LANE,HARLESTON,STOWMARKET,M..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
113615,{C18F412B-A9C2-81A6-E053-6B04A8C0AD18},46000,2021-01-29 00:00,PR1 2AQ,F,N,L,CUBIC,30,BIRLEY STREET,NaN,PRESTON,PRESTON,LANCASHIRE,B,D,United Kingdom,"CUBIC,30,BIRLEY STREET,PRESTON,PRESTON,PR1 2AQ..."
113616,{C18F412B-AA70-81A6-E053-6B04A8C0AD18},50000,2021-01-25 00:00,PE21 6DJ,T,N,F,1,NaN,STILL LANE,NaN,BOSTON,BOSTON,LINCOLNSHIRE,B,D,United Kingdom,"1,STILL LANE,BOSTON,BOSTON,PE21 6DJ,LINCOLNSHI..."
113617,{C18F412B-AEAA-81A6-E053-6B04A8C0AD18},95000,2021-02-19 00:00,NE23 6JH,T,N,F,41,NaN,ANTON PLACE,NaN,CRAMLINGTON,NORTHUMBERLAND,NORTHUMBERLAND,B,D,United Kingdom,"41,ANTON PLACE,CRAMLINGTON,NORTHUMBERLAND,NE23..."
113618,{C18F412B-B286-81A6-E053-6B04A8C0AD18},174000,2021-01-13 00:00,ST6 1DQ,O,N,F,2,NaN,MOORLAND ROAD,BURSLEM,STOKE-ON-TRENT,STOKE-ON-TRENT,STOKE-ON-TRENT,B,D,United Kingdom,"2,MOORLAND ROAD,BURSLEM,STOKE-ON-TRENT,STOKE-O..."


In [7]:
#df.to_csv('./address.csv')

In [8]:
df = df.head(50)
df

,TUID,Price,Date,Postcode,Property_Type,OldNew,Duration,PAON,SAON,Street,Locality,City,District,County,PPDtype,RecordStatus,Country,Address
0,{C6209F5F-241F-295E-E053-6C04A8C0DDCC},310000,2004-09-08 00:00,RM12 4DP,T,N,F,51,NaN,BANCROFT CHASE,NaN,HORNCHURCH,HAVERING,GREATER LONDON,A,A,United Kingdom,"51,BANCROFT CHASE,HORNCHURCH,HAVERING,RM12 4DP..."
1,{C6209F5F-4752-295E-E053-6C04A8C0DDCC},440000,2004-04-08 00:00,WD25 0DY,D,N,F,22,NaN,SHEEPCOT DRIVE,NaN,WATFORD,WATFORD,HERTFORDSHIRE,A,A,United Kingdom,"22,SHEEPCOT DRIVE,WATFORD,WATFORD,WD25 0DY,HER..."
2,{C6209F5F-497D-295E-E053-6C04A8C0DDCC},415000,2004-03-26 00:00,AL1 1NL,T,N,F,6,NaN,RYDER SEED MEWS,NaN,ST ALBANS,ST ALBANS,HERTFORDSHIRE,A,A,United Kingdom,"6,RYDER SEED MEWS,ST ALBANS,ST ALBANS,AL1 1NL,..."
3,{C6209F5F-9679-295E-E053-6C04A8C0DDCC},540000,2004-10-12 00:00,SW7 4DA,F,N,L,29 - 31,FLAT 8,COURTFIELD ROAD,NaN,LONDON,KENSINGTON AND CHELSEA,GREATER LONDON,A,A,United Kingdom,"29 - 31,FLAT 8,COURTFIELD ROAD,LONDON,KENSINGT..."
4,{C6209F5F-C5BA-295E-E053-6C04A8C0DDCC},570000,2004-08-12 00:00,IP14 3JH,D,N,F,NELSONS,NaN,MOORBRIDGE LANE,HARLESTON,STOWMARKET,MID SUFFOLK,SUFFOLK,A,A,United Kingdom,"NELSONS,MOORBRIDGE LANE,HARLESTON,STOWMARKET,M..."
5,{C6209F5F-FB5A-295E-E053-6C04A8C0DDCC},32750,2004-05-07 00:00,SA38 9JN,F,N,L,PENLAN HOLIDAY VILLAGE,34,NaN,CENARTH,NEWCASTLE EMLYN,CARMARTHENSHIRE,CARMARTHENSHIRE,A,A,United Kingdom,"PENLAN HOLIDAY VILLAGE,34,CENARTH,NEWCASTLE EM..."
6,{C6209F5F-00B5-295E-E053-6C04A8C0DDCC},51337,2004-03-31 00:00,SA62 5SJ,T,N,F,60,NaN,ST DAVIDS ROAD,LETTERSTON,HAVERFORDWEST,PEMBROKESHIRE,PEMBROKESHIRE,A,A,United Kingdom,"60,ST DAVIDS ROAD,LETTERSTON,HAVERFORDWEST,PEM..."
7,{C6209F5F-00F5-295E-E053-6C04A8C0DDCC},46000,2004-09-24 00:00,CH8 8LY,D,N,F,GRAIG BACH,NaN,FFORDD Y GRAIG,LIXWM,HOLYWELL,FLINTSHIRE,FLINTSHIRE,A,A,United Kingdom,"GRAIG BACH,FFORDD Y GRAIG,LIXWM,HOLYWELL,FLINT..."
8,{C6209F5F-329E-295E-E053-6C04A8C0DDCC},220000,2004-07-26 00:00,RM16 6RZ,T,N,F,37,NaN,CAREW CLOSE,CHAFFORD HUNDRED,GRAYS,THURROCK,THURROCK,A,A,United Kingdom,"37,CAREW CLOSE,CHAFFORD HUNDRED,GRAYS,THURROCK..."
9,{C6209F5F-8CC9-295E-E053-6C04A8C0DDCC},123000,2004-07-02 00:00,PR9 9HR,F,Y,L,"PRINCESS COURT, 39",FLAT 9,ALEXANDRA ROAD,NaN,SOUTHPORT,SEFTON,MERSEYSIDE,A,A,United Kingdom,"PRINCESS COURT, 39,FLAT 9,ALEXANDRA ROAD,SOUTH..."


In [9]:
from geopy.extra.rate_limiter import RateLimiter
geocode = RateLimiter(locator.geocode, min_delay_seconds=1)
df['location'] = df['Address'].apply(geocode)
df['point'] = df['location'].apply(lambda loc: tuple(loc.point) if loc else None)

C:\Users\METE\AppData\Local\Temp/ipykernel_6712/894746365.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['location'] = df['Address'].apply(geocode)
C:\Users\METE\AppData\Local\Temp/ipykernel_6712/894746365.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['point'] = df['location'].apply(lambda loc: tuple(loc.point) if loc else None)


In [10]:
print(df)

                                      TUID   Price              Date  \
0   {C6209F5F-241F-295E-E053-6C04A8C0DDCC}  310000  2004-09-08 00:00   
1   {C6209F5F-4752-295E-E053-6C04A8C0DDCC}  440000  2004-04-08 00:00   
2   {C6209F5F-497D-295E-E053-6C04A8C0DDCC}  415000  2004-03-26 00:00   
3   {C6209F5F-9679-295E-E053-6C04A8C0DDCC}  540000  2004-10-12 00:00   
4   {C6209F5F-C5BA-295E-E053-6C04A8C0DDCC}  570000  2004-08-12 00:00   
5   {C6209F5F-FB5A-295E-E053-6C04A8C0DDCC}   32750  2004-05-07 00:00   
6   {C6209F5F-00B5-295E-E053-6C04A8C0DDCC}   51337  2004-03-31 00:00   
7   {C6209F5F-00F5-295E-E053-6C04A8C0DDCC}   46000  2004-09-24 00:00   
8   {C6209F5F-329E-295E-E053-6C04A8C0DDCC}  220000  2004-07-26 00:00   
9   {C6209F5F-8CC9-295E-E053-6C04A8C0DDCC}  123000  2004-07-02 00:00   
10  {C6209F5F-F5F2-295E-E053-6C04A8C0DDCC}  160000  2004-12-22 00:00   
11  {C6209F60-07F6-295E-E053-6C04A8C0DDCC}   84950  2004-05-26 00:00   
12  {C6209F60-0801-295E-E053-6C04A8C0DDCC}   79950  2004-05-26 0

In [11]:
df['point'][0][0]

51.55925

In [12]:
# split point column into latitude, longitude and altitude columns
df[['latitude', 'longitude', 'altitude']] = pd.DataFrame(df['point'].tolist(), index=df.index)
df.head()

C:\Users\METE\anaconda3\envs\test\lib\site-packages\pandas\core\frame.py:3636: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[k1] = value[k2]


,TUID,Price,Date,Postcode,Property_Type,OldNew,Duration,PAON,SAON,Street,...,County,PPDtype,RecordStatus,Country,Address,location,point,latitude,longitude,altitude
0,{C6209F5F-241F-295E-E053-6C04A8C0DDCC},310000,2004-09-08 00:00,RM12 4DP,T,N,F,51,NaN,BANCROFT CHASE,...,GREATER LONDON,A,A,United Kingdom,"51,BANCROFT CHASE,HORNCHURCH,HAVERING,RM12 4DP...","(51 Bancroft Chase, Hornchurch, RM12 4DP, Unit...","(51.55925, 0.18573, 0.0)",51.559250,0.18573,0.0
1,{C6209F5F-4752-295E-E053-6C04A8C0DDCC},440000,2004-04-08 00:00,WD25 0DY,D,N,F,22,NaN,SHEEPCOT DRIVE,...,HERTFORDSHIRE,A,A,United Kingdom,"22,SHEEPCOT DRIVE,WATFORD,WATFORD,WD25 0DY,HER...","(22 Sheepcot Drive, Watford, WD25 0DY, United ...","(51.6897123, -0.3930805, 0.0)",51.689712,-0.39308,0.0
2,{C6209F5F-497D-295E-E053-6C04A8C0DDCC},415000,2004-03-26 00:00,AL1 1NL,T,N,F,6,NaN,RYDER SEED MEWS,...,HERTFORDSHIRE,A,A,United Kingdom,"6,RYDER SEED MEWS,ST ALBANS,ST ALBANS,AL1 1NL,...","(Ryder Seed Mews, St Albans, AL1 1NL, United K...","(51.7493905, -0.3394905, 0.0)",51.749390,-0.33949,0.0
3,{C6209F5F-9679-295E-E053-6C04A8C0DDCC},540000,2004-10-12 00:00,SW7 4DA,F,N,L,29 - 31,FLAT 8,COURTFIELD ROAD,...,GREATER LONDON,A,A,United Kingdom,"29 - 31,FLAT 8,COURTFIELD ROAD,LONDON,KENSINGT...","(8 Courtfield Road, London, SW7 4, United King...","(51.49346, -0.18468, 0.0)",51.493460,-0.18468,0.0
4,{C6209F5F-C5BA-295E-E053-6C04A8C0DDCC},570000,2004-08-12 00:00,IP14 3JH,D,N,F,NELSONS,NaN,MOORBRIDGE LANE,...,SUFFOLK,A,A,United Kingdom,"NELSONS,MOORBRIDGE LANE,HARLESTON,STOWMARKET,M...","(Moorbridge Lane, Stowmarket, IP14 3JH, United...","(52.2122, 0.94932, 0.0)",52.212200,0.94932,0.0


In [13]:
df.columns

Index(['TUID', 'Price', 'Date', 'Postcode', 'Property_Type', 'OldNew',
       'Duration', 'PAON', 'SAON', 'Street', 'Locality', 'City', 'District',
       'County', 'PPDtype', 'RecordStatus', 'Country', 'Address', 'location',
       'point', 'latitude', 'longitude', 'altitude'],
      dtype='object')

In [14]:
df = df.drop(['Postcode', 'PAON', 'SAON', 'Street','Locality', 'City', 'District', 'County', 'Country', 'location', 'point', 'altitude'], axis=1)
df.to_csv('./address.csv')

In [15]:
df.head()

,TUID,Price,Date,Property_Type,OldNew,Duration,PPDtype,RecordStatus,Address,latitude,longitude
0,{C6209F5F-241F-295E-E053-6C04A8C0DDCC},310000,2004-09-08 00:00,T,N,F,A,A,"51,BANCROFT CHASE,HORNCHURCH,HAVERING,RM12 4DP...",51.559250,0.18573
1,{C6209F5F-4752-295E-E053-6C04A8C0DDCC},440000,2004-04-08 00:00,D,N,F,A,A,"22,SHEEPCOT DRIVE,WATFORD,WATFORD,WD25 0DY,HER...",51.689712,-0.39308
2,{C6209F5F-497D-295E-E053-6C04A8C0DDCC},415000,2004-03-26 00:00,T,N,F,A,A,"6,RYDER SEED MEWS,ST ALBANS,ST ALBANS,AL1 1NL,...",51.749390,-0.33949
3,{C6209F5F-9679-295E-E053-6C04A8C0DDCC},540000,2004-10-12 00:00,F,N,L,A,A,"29 - 31,FLAT 8,COURTFIELD ROAD,LONDON,KENSINGT...",51.493460,-0.18468
4,{C6209F5F-C5BA-295E-E053-6C04A8C0DDCC},570000,2004-08-12 00:00,D,N,F,A,A,"NELSONS,MOORBRIDGE LANE,HARLESTON,STOWMARKET,M...",52.212200,0.94932


In [16]:
df.latitude.isnull().sum()

0

In [17]:
df = df[pd.notnull(df["latitude"])]

In [18]:
map1 = folium.Map(
    location=[52.474, -1.000],
    tiles='cartodbpositron',
    zoom_start=7,
)

In [19]:
df.apply(lambda row:folium.CircleMarker(location=[row["latitude"], row["longitude"]]).add_to(map1), axis=1)
map1

In [20]:
map1.save("map1.html")

In [21]:
folium_map = folium.Map(location=[52.474, -1.000],
                        zoom_start=7,
                        tiles='CartoDB dark_matter')


FastMarkerCluster(data=list(zip(df['latitude'].values, df['longitude'].values))).add_to(folium_map)
folium.LayerControl().add_to(folium_map)
folium_map

In [22]:
folium_map.save("map2.html")